In [2]:
import numpy as np
from nltk.corpus import stopwords
import nltk
from collections import Counter
from numba import njit, prange
import time, pickle, os
from scipy.sparse.linalg import svds
from scipy.sparse import csr

# Load Data

In [2]:
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\juxia\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
with open("wiki-text.txt", 'r') as file:
    wc = file.read()
wc = wc.split()

In [4]:
len(wc)

124301826

In [5]:
print(wc[:20])

['anarchism', 'originated', 'as', 'a', 'term', 'of', 'abuse', 'first', 'used', 'against', 'early', 'working', 'class', 'radicals', 'including', 'the', 'diggers', 'of', 'the', 'english']


# Filter volcabulary
-Remove words that appear less than n times (here n = 500). 

-Remove all words that appear in the stopwords list of nltk package.

In [6]:
wc_count = Counter(wc)

In [7]:
vocab_dict = {}
for vocab in wc_count:
    if wc_count[vocab] > 500 and vocab not in stop_words:
        vocab_dict[vocab] = wc_count[vocab]
d = len(vocab_dict)
d

13201

In [8]:
# Remove words not in vocabulary
def remove_excess_words(wc):
    removed = []
    for word in wc:
        if word in vocab_dict:
            removed.append(word)
    return removed        

In [9]:
removed_wc = remove_excess_words(wc)

In [10]:
print(removed_wc[:20])

['anarchism', 'originated', 'term', 'abuse', 'first', 'used', 'early', 'working', 'class', 'radicals', 'including', 'english', 'revolution', 'sans', 'french', 'revolution', 'whilst', 'term', 'still', 'used']


In [11]:
len(removed_wc)

71618337

In [12]:
# Free up memory
del wc

In [20]:
# Save dictionary
# pickle.dump(removed_wc, open("removed_wc.p", "wb" ) )
pickle.dump(vocab_dict, open("vocab_dict.p", "wb" ) )

In [18]:
# Load dictionary
# removed_wc = pickle.load(open("removed_wc.p", "rb" ) )
vocab_dict = pickle.load(open("vocab_dict.p", "rb" ) )

# Compute PMI Matrix
$
M_{ij} = \log\left( \frac{(N^p(w_i, w_j)+1)\cdot N(S^p)}{N^p(w_i)\cdot N^p(w_j)} \right)
$

In [20]:
# Create a vocabulary index look-up table. 
# We exploit the fact that dict.keys are ordered in Python 3.
def return_vilut(vocab_dict):
    vocab_index_look_up_table = {}
    vocab_list = list(vocab_dict.keys())
    for i in range(len(vocab_dict.keys())):
        vocab_index_look_up_table[vocab_list[i]] = i
    return vocab_index_look_up_table

In [21]:
vilut = return_vilut(vocab_dict)

# Sample look-up table, real look up table is a dict
[(i, vilut[i]) for i in list(vilut.keys())[:5]]

[('anarchism', 0), ('originated', 1), ('term', 2), ('abuse', 3), ('first', 4)]

In [5]:
def count_word_occurences(wc, vilut):
    size = 2
    d = len(vilut)
    n = len(wc)
    # N^p(w_i, w_j)
    word_pair_count = np.zeros((d, d),dtype="int64")
    # N^p(w)
    word_count = np.zeros(d,dtype="int64")
    # N(S^p)
    NSp = 0
    t1 = time.time()
    for i in range(n):
        if i%10**7==0:
            print(f"{i}/{n} completed, {round(time.time()-t1, 2)}s elapsed")
        neighbours = list(range(max(0, i-size), min(i+size+1, n)))
        neighbours.remove(i)
        for j in neighbours:
            ind_i = vilut[wc[i]]
            ind_j = vilut[wc[j]]
            word_pair_count[ind_i, ind_j] += 1
            word_count[ind_j] += 1
            NSp += 1
    return word_count, word_pair_count, NSp

In [6]:
word_count, word_pair_count, NSp = count_word_occurences(removed_wc,vilut)

0/71618337 completed, 0.0s elapsed
10000000/71618337 completed, 55.39s elapsed
20000000/71618337 completed, 107.81s elapsed
30000000/71618337 completed, 161.29s elapsed
40000000/71618337 completed, 210.53s elapsed
50000000/71618337 completed, 261.34s elapsed
60000000/71618337 completed, 313.4s elapsed
70000000/71618337 completed, 365.29s elapsed


In [7]:
M = np.log((word_pair_count+1) * NSp / (word_count.reshape(-1,1) @ word_count.reshape(1,-1)))

In [8]:
# Save the precious data
pickle.dump(word_pair_count, open("word_pair_count.p", "wb" ))
pickle.dump(word_count, open("word_count.p", "wb" ) )
pickle.dump(M, open("M.p", "wb" ) )

In [9]:
M

array([[ 7.77524551,  2.66743035,  1.76739777, ...,  3.28795218,
         1.78547095,  2.47968366],
       [ 2.66743035,  1.71458438,  3.03375528, ...,  1.92964109,
         0.42715986,  1.12137257],
       [ 1.76739777,  3.03375528,  2.31725125, ..., -0.5798294 ,
        -2.08231063, -0.69495074],
       ...,
       [ 3.28795218,  1.92964109, -0.5798294 , ...,  3.24331009,
         1.74082887,  2.43504158],
       [ 1.78547095,  0.42715986, -2.08231063, ...,  1.74082887,
         0.23834764,  0.93256035],
       [ 2.47968366,  1.12137257, -0.69495074, ...,  2.43504158,
         0.93256035,  1.62677306]])

# Factorize the PMI matrix to obtain word embeddings

In [3]:
# Load
M = pickle.load(open("M.p", "rb" ) )

In [4]:
U, s, V =svds(M, k=50)

In [11]:
W = U @ np.diag(s**(1/2))

In [26]:
W.shape

(13201, 50)

In [13]:
# Save
pickle.dump(W, open("W.p", "wb" ) )

# Application: Find closest word

In [32]:
def get_context(W, vocab_dict, vilut, word, n):
    """
    Returns n words which appear around word with highest probability."""
    v_w = W[vilut[word]]
    n_max_ind = np.argsort(-W @ v_w)[:n]
    vocab = list(vocab_dict.keys())
    return [vocab[i] for i in n_max_ind]

In [33]:
get_context(W, vocab_dict, vilut, "physics", 5)

['physics', 'science', 'mathematics', 'theory', 'mathematical']

In [34]:
get_context(W, vocab_dict, vilut, "republican", 5)

['republican', 'president', 'election', 'party', 'democratic']

In [35]:
get_context(W, vocab_dict, vilut, "einstein", 5)

['lorentz', 'planck', 'leibniz', 'gauss', 'einstein']

In [37]:
get_context(W, vocab_dict, vilut, "algebra", 5)

['theorem', 'algebra', 'frac', 'finite', 'vector']

In [38]:
get_context(W, vocab_dict, vilut, "fish", 5)

['species', 'food', 'fish', 'fruit', 'plants']

# Application: Solve analogy

In [39]:
def solve_analogies(X, Y, Z, W, vocab_dict, vilut,n):
    """
    Returns n words that solve the analogy X : Y | Z : ? 
    """
    v_w = W[vilut[Y]] - W[vilut[X]] + W[vilut[Z]]
    n_max_ind = np.argsort(-W @ v_w)[:n]
    vocab = list(vocab_dict.keys())
    return [vocab[i] for i in n_max_ind]

In [41]:
solve_analogies("republican", "democrat", "conservative", W, vocab_dict, vilut,5)

['conservative', 'liberal', 'conservatives', 'democrats', 'views']

In [44]:
solve_analogies("france", "paris", "england", W, vocab_dict, vilut,5)

['london', 'england', 'st', 'college', 'york']

In [68]:
solve_analogies("car", "road", "boat", W, vocab_dict, vilut,5)

['river', 'north', 'located', 'south', 'west']

In [93]:
solve_analogies("america", "capitalist", "soviet", W, vocab_dict, vilut,5)

['soviet', 'german', 'forces', 'military', 'communist']